In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load the GLUE benchmark dataset, specify which task to load (e.g., 'sst2' for sentiment analysis)
dataset = load_dataset("glue", "sst2")

# Check the dataset
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})


In [ ]:
!pip install transformers datasets

In [ ]:
print(dataset['train'][0])

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}


In [ ]:
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split
import torch

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
dataset = load_dataset("glue", "sst2")

# Limit the dataset to 8,000 rows from the training set
train_data = dataset['train'].select(range(8000))

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['sentence'], truncation=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenized_train_data = train_data.map(tokenize_function, batched=True)

# Load the DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_data = dataset['train'].select(range(8000))

# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['sentence'], truncation=True)

# Tokenize the dataset
tokenized_train_data = train_data.map(tokenize_function, batched=True)

# Use Hugging Face's `train_test_split` method to split the dataset
split_datasets = tokenized_train_data.train_test_split(test_size=0.1)

# Get the train and validation datasets
train_dataset = split_datasets['train']
val_dataset = split_datasets['test']

# Convert datasets into PyTorch format
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir='./logs',
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.233843
2,0.335600,0.242674
3,0.173500,0.294264


TrainOutput(global_step=1350, training_loss=0.21650087144639757, metrics={'train_runtime': 891.1711, 'train_samples_per_second': 24.238, 'train_steps_per_second': 1.515, 'total_flos': 196950044558016.0, 'train_loss': 0.21650087144639757, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.2942636013031006, 'eval_runtime': 0.924, 'eval_samples_per_second': 865.759, 'eval_steps_per_second': 54.11, 'epoch': 3.0}


In [ ]:
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/added_tokens.json')

In [ ]:
from transformers import pipeline

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('./results')
model = DistilBertForSequenceClassification.from_pretrained('./results')

In [ ]:
dataset = load_dataset("glue", "sst2")

# Use the first 5 rows from the validation set as an example (you can adjust this as needed)
val_data = dataset['validation'].select(range(5))

# Tokenize the validation data (you can adjust this to your new data as well)
def tokenize_function(examples):
    return tokenizer(examples['sentence'], truncation=True)

# Tokenize validation data
tokenized_val_data = val_data.map(tokenize_function, batched=True)

# Convert to PyTorch tensors
tokenized_val_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, DataCollatorWithPadding
from datasets import load_dataset

# Load the tokenizer and fine-tuned model (assuming it is saved in './results')
tokenizer = DistilBertTokenizer.from_pretrained('./results')
model = DistilBertForSequenceClassification.from_pretrained('./results')

dataset = load_dataset("glue", "sst2")

val_data = dataset['validation'].select(range(5))

def tokenize_function(examples):
    return tokenizer(examples['sentence'], truncation=True)

tokenized_val_data = val_data.map(tokenize_function, batched=True)
tokenized_val_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    data_collator=data_collator
)

predictions = trainer.predict(tokenized_val_data)

predicted_labels = predictions.predictions.argmax(-1)
print("Predicted labels:", predicted_labels)

for i, sentence in enumerate(val_data['sentence']):
    print(f"Sentence: {sentence}\nPredicted Label: {predicted_labels[i]}\n")

Predicted labels: [1 1 1 0 1]
Sentence: it 's a charming and often affecting journey . 
Predicted Label: 1

Sentence: unflinchingly bleak and desperate 
Predicted Label: 1

Sentence: allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . 
Predicted Label: 1

Sentence: the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . 
Predicted Label: 0

Sentence: it 's slow -- very , very slow . 
Predicted Label: 1



In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments

# Define a function to compute metrics for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Update the Trainer definition to include `compute_metrics`
trainer = Trainer(
    model=DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics  # Add compute_metrics here
)

# Train and evaluate
trainer.train()
results = trainer.evaluate()

# Print evaluation results
print("Evaluation metrics:", results)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.231639,0.903750,0.918691,0.927505,0.910042
2,0.338000,0.242944,0.920000,0.933194,0.931250,0.935146
3,0.172100,0.275485,0.927500,0.939076,0.943038,0.935146


Evaluation metrics: {'eval_loss': 0.2754853367805481, 'eval_accuracy': 0.9275, 'eval_f1': 0.9390756302521008, 'eval_precision': 0.9430379746835443, 'eval_recall': 0.9351464435146444, 'eval_runtime': 1.03, 'eval_samples_per_second': 776.681, 'eval_steps_per_second': 48.543, 'epoch': 3.0}
